# 1. Cài đặt và chuẩn bị môi trường

In [ ]:
!pip uninstall -y transformers
!pip uninstall -y tokenizers
!pip uninstall -y huggingface_hub

!pip install transformers==4.49.0
!pip install underthesea
!pip install vncorenlp
!pip install scikit-learn
!pip install pyarrow

import pandas as pd
import os
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from torch.nn.functional import softmax
import numpy as np

print("Import Successfull")

# 2. Dataset class và utility functions

In [ ]:
# Dùng tokenizer từ PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

class HateDataset(Dataset):
    def __init__(self, texts, labels, max_len=128):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)

print("Done setup hatedataset")

In [ ]:
voz_df = pd.read_csv("/kaggle/input/hsdcomment/voz-hsd.csv", engine="pyarrow")
voz_df = voz_df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle cho ngẫu nhiên

chunk_size = 1_000_000 
os.makedirs("/kaggle/working/voz_chunks", exist_ok=True)

for i in range(0, len(voz_df), chunk_size):
    chunk = voz_df.iloc[i:i+chunk_size]
    chunk.to_csv(f"/kaggle/working/voz_chunks/voz_chunk_{i//chunk_size}.csv", index=False)

print(f"Đã chia thành {len(os.listdir('/kaggle/working/voz_chunks'))} file.")

# 3. Load và kiểm tra dữ liệu

In [ ]:
print("📁 Đang load datasets...")
voz_chunks_dir = "/kaggle/input/vozhsd-chunks"

def analyze_chunk_distribution():
    chunk_files = sorted(os.listdir(voz_chunks_dir))
    distributions = []
    total_voz_samples = 0  # ✅ Thêm biến đếm tổng
    
    for i, chunk_file in enumerate(chunk_files):
        chunk_df = pd.read_csv(f"{voz_chunks_dir}/{chunk_file}")
        total_voz_samples += len(chunk_df)  # ✅ Cộng dồn
        
        # ✅ Chỉ show dữ liệu mẫu của chunk đầu tiên
        if i == 0:
            print(f"\n📄 Sample data from {chunk_file}:")
            print(chunk_df.head())
            
        class_dist = chunk_df['labels'].value_counts(normalize=True)
        hate_ratio = class_dist.get(1, 0)
        distributions.append({
            'chunk': chunk_file,
            'total_samples': len(chunk_df),
            'hate_ratio': hate_ratio,
            'clean_ratio': 1 - hate_ratio
        })
        print(f"{chunk_file}: {len(chunk_df)} samples, Hate: {hate_ratio:.3f}, Clean: {1 - hate_ratio:.3f}")
    
    # ✅ Thêm tổng kết VoZ dataset
    print(f"\n🔢 TỔNG KẾT VoZ DATASET:")
    print(f"   📊 Tổng chunks: {len(chunk_files)}")
    print(f"   📊 Tổng samples: {total_voz_samples:,}")
    
    hate_ratios = [d['hate_ratio'] for d in distributions]
    std_dev = np.std(hate_ratios)
    print(f"   📊 Hate ratio std: {std_dev:.4f}")
    
    return distributions

distributions = analyze_chunk_distribution()

# Load ViHSD datasets
train_vihsd = pd.read_csv("/kaggle/input/hsdcomment/train.csv")
val_vihsd = pd.read_csv("/kaggle/input/hsdcomment/dev.csv")
test_vihsd = pd.read_csv("/kaggle/input/hsdcomment/test.csv")

# ✅ Thêm thông tin số lượng ngay khi load
print(f"\n📁 LOADED ViHSD DATASETS:")
print(f"   📊 Train: {len(train_vihsd):,} samples")
print(f"   📊 Dev:   {len(val_vihsd):,} samples")
print(f"   📊 Test:  {len(test_vihsd):,} samples")
print(f"   📊 Total: {len(train_vihsd) + len(val_vihsd) + len(test_vihsd):,} samples")

# Gộp nhãn OFFENSIVE + HATE thành 1
for df in [train_vihsd, val_vihsd, test_vihsd]:
    df['label_id'] = df['label_id'].replace({2: 1})

# ✅ Hiển thị vài dòng đầu mỗi tập
print("\n📄 Sample from ViHSD train.csv:")
print(train_vihsd.head())

print("\n📄 Sample from ViHSD dev.csv:")
print(val_vihsd.head())

print("\n📄 Sample from ViHSD test.csv:")
print(test_vihsd.head())

# ✅ Kiểm tra phân bố class trong TẤT CẢ 3 tập
print(f"\n📊 ViHSD CLASS DISTRIBUTION:")

for dataset_name, dataset in [("Train", train_vihsd), ("Dev", val_vihsd), ("Test", test_vihsd)]:
    class_dist = dataset['label_id'].value_counts(normalize=True)
    class_count = dataset['label_id'].value_counts()
    
    print(f"\n   {dataset_name} ({len(dataset):,} samples):")
    print(f"     Clean (0): {class_count.get(0, 0):,} samples ({class_dist.get(0, 0):.3f})")
    print(f"     Hate  (1): {class_count.get(1, 0):,} samples ({class_dist.get(1, 0):.3f})")

# 4. Train phase 1: Pretrain on noisy data (VOZ-HSD)

In [ ]:
chunk_dir = "/kaggle/input/vozhsd-chunks"
save_dir = "/kaggle/working/pretrain_voz_continued"
base_lr = 5e-5

min_chunk_idx = 8
max_chunk_idx = 10

chunk_files = sorted(os.listdir(chunk_dir))
selected_chunks = chunk_files[min_chunk_idx : max_chunk_idx + 1]

# Load model pretrain trước đó
pretrained_model_path = "/kaggle/input/pretrain_vozz/transformers/default/1"
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_path, num_labels=2)
print("Đã preload xong pretrain_model, bắt đầu train\n")

for chunk_i, chunk_file in enumerate(selected_chunks, start=min_chunk_idx):
    print(f"🔄 Đang train chunk {chunk_i}: {chunk_file}")
    
    chunk_df = pd.read_csv(os.path.join(chunk_dir, chunk_file))
    train_dataset = HateDataset(chunk_df["texts"], chunk_df["labels"])

    training_args = TrainingArguments(
        output_dir=f"{save_dir}/chunk_{chunk_i}",
        per_device_train_batch_size=32,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=base_lr,
        save_strategy="no",
        logging_dir=f"./logs/chunk_{chunk_i}",
        logging_steps=500,
        logging_first_step=True,
        save_total_limit=1,
        dataloader_num_workers=2,
        dataloader_pin_memory=True,
        disable_tqdm=False,
        report_to="none",
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )

    trainer.train()

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Đã hoàn thành pretrain từ chunk {min_chunk_idx} đến chunk {max_chunk_idx}\n")
print(f"✅ Đã lưu model và tokenizer vào {save_dir}")


# 5. Train phase 2: Fine-tune on clean data (ViHSD)

In [ ]:
from transformers import EarlyStoppingCallback
from transformers import AutoConfig
from transformers import RobertaForSequenceClassification
import torch
import torch.nn as nn

# ===== Custom Model sử dụng class weights =====
class WeightedLossModel(RobertaForSequenceClassification):
    def __init__(self, config, class_weights=None):
        super().__init__(config)
        self.class_weights = class_weights

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        # Gọi forward gốc chỉ với input_ids và attention_mask
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits = outputs.logits
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
            loss = loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}

    @classmethod
    def from_pretrained_with_weights(cls, model_path, class_weights, **kwargs):
        config = AutoConfig.from_pretrained(model_path, num_labels=2)
        model = cls(config=config, class_weights=class_weights)
        pretrained = RobertaForSequenceClassification.from_pretrained(model_path)
        model.load_state_dict(pretrained.state_dict(), strict=False)
        return model


model_path = "/kaggle/input/pretrain_vozz/transformers/default/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load dữ liệu VIHSD đã chia sẵn
train_vihsd = pd.read_csv("/kaggle/input/hsdcomment/train.csv")
val_vihsd = pd.read_csv("/kaggle/input/hsdcomment/dev.csv")
test_vihsd = pd.read_csv("/kaggle/input/hsdcomment/test.csv")

# Gộp nhãn OFFENSIVE + HATE thành 1 trong cả 3 bộ dữ liệu
for df in [train_vihsd, val_vihsd, test_vihsd]:
    df['label_id'] = df['label_id'].replace({2: 1})

# Tính class weight 
counts = train_vihsd['label_id'].value_counts().to_dict()
total = sum(counts.values())
class_weights = torch.tensor([
    total / (2 * counts.get(0, 1)),
    total / (2 * counts.get(1, 1))
], dtype=torch.float)

train_dataset = HateDataset(train_vihsd["free_text"], train_vihsd["label_id"])
val_dataset = HateDataset(val_vihsd["free_text"], val_vihsd["label_id"])

counts = train_vihsd['label_id'].value_counts().to_dict()
total = sum(counts.values())
class_weights = torch.tensor([
    total / (2 * counts.get(0, 1)),
    total / (2 * counts.get(1, 1))
], dtype=torch.float)

model = WeightedLossModel.from_pretrained_with_weights(model_path, class_weights)

training_args = TrainingArguments(
    output_dir="/kaggle/working/finetune_vihsd",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_finetune",
    logging_first_step=True,
    logging_steps=500,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=1,
    fp16=True,
    greater_is_better=False,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    disable_tqdm=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

print("✅ Bắt đầu fine-tune on ViHSD với class weights")
trainer.train()

# 6. Lưu mô hình để dùng trong API

In [ ]:
import zipfile
import os

final_model_path = "./hate_speech_model"
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"✅ Đã lưu model sau khi finetune tại: {final_model_path}")

In [ ]:
# Fix the plotting code
logs = trainer.state.log_history

# Separate training and evaluation logs
train_logs = [log for log in logs if "loss" in log and "epoch" in log and "eval_loss" not in log]
eval_logs = [log for log in logs if "eval_loss" in log]

# Extract data for training plot
train_loss = [log["loss"] for log in train_logs]
train_epochs = [log["epoch"] for log in train_logs]

# Extract data for evaluation plot
eval_loss = [log["eval_loss"] for log in eval_logs]
eval_epochs = [log["epoch"] for log in eval_logs]

# Plot both curves
plt.figure(figsize=(10,5))
plt.plot(train_epochs, train_loss, label="Train Loss")
plt.plot(eval_epochs, eval_loss, label="Eval Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Train/Eval Loss per Epoch")
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Tạo dataset từ tập test
test_dataset = HateDataset(test_vihsd["free_text"], test_vihsd["label_id"])

# Hiển thị số lượng mẫu
print(f"Số lượng mẫu trong tập test: {len(test_dataset)}")

# Dự đoán
predictions = trainer.predict(test_dataset)

# logits -> nhãn dự đoán
pred_labels = predictions.predictions.argmax(axis=1)

# Nhãn thật
true_labels = predictions.label_ids

# Tính accuracy
acc = accuracy_score(true_labels, pred_labels)

print(f"🎯 Accuracy trên tập test: {acc * 100:.2f}%")

# In báo cáo chi tiết
print("\n📋 Classification Report:")
print(classification_report(true_labels, pred_labels, digits=4))

# 7. Dự đoán test nhanh

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Load lại model và tokenizer đã lưu
model_path = "./hate_speech_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  # Đưa model về chế độ evaluation (rất quan trọng khi inference)

# Text cần dự đoán
text = "Mình rất trân trọng công sức của bạn, thật sự là vậy. Nhưng mà bài viết này thì đúng nghĩa rác rưởi chó đẻ, thậm chí còn xúc phạm trí tuệ người đọc. Mong bạn dành thời gian học lại trước khi chia sẻ ý kiến ra cộng đồng vì đó là điều rất quan trọng há"

# Tokenize
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Model inference
with torch.no_grad():  # Không cần tính toán gradient trong lúc dự đoán
    outputs = model(**inputs)

# Softmax để lấy xác suất
probs = softmax(outputs.logits, dim=1)
label_id = torch.argmax(probs, dim=1).item()
confidence = probs[0][label_id].item() * 100

# Mapping id -> nhãn
label_map = {0: "CLEAN", 1: "HATE"}

# In kết quả
for i, label in label_map.items():
    print(f"{label}: {probs[0][i].item() * 100:.2f}%")

# Nếu vẫn muốn in thêm nhãn được dự đoán cao nhất
print("\nKết luận dự đoán:")
print("Kết quả:", label_map[label_id])
print("Tỷ lệ dự đoán:", f"{confidence:.2f}%")